# Enzyme Commission Class with Ligands

# Overview

**Questions**

1. How are enzymes classified?
2. How can I search the PDB for ligands that bind to a specific enzyme class?

**Learning Objectives**

1. Understand the hierarchical classification of enzymes using the Enzyme Commission (EC) system.
2. Use the RCSB Search API to find ligands that bind to a specific enzyme class.

Enzymes ar Enzymes are biological catalysts and most enzymes are proteins (at least that's our current thinking). To systematize the study of enzymes, IUPAC (the International Union of Pure and Applied Chemistry) has organized enzymes in a hierarchical class structure, with 7 top level classes and a total of 4 levels in the hierarchy.

These classes are called the Enzyme Commission (EC) classes, and the hierarchy is called the EC system. Each enzyme is assigned an EC number based on the type of reaction it catalyzes. In this notebook, we will use the RCSB Search API to find ligands that bind to a specific enzyme class.

# The 7 Enzyme Classes

The BRENDA Enzyme Database contains detailed information about enzymes and includes a browser tree for enzyme classes. The 7 major classes of enzymes on the top level of the tree are

1. Oxidoreductases
2. Transferases
3. Hydrolases
4. Lyases
5. Isomerases
6. Ligases
7. Translocases

On the BRENDA Home Page, you can search for an enzyme by name and find its Enzyme Commision (EC) number, and a lot more detail as well. The first image below is the search for the enzyme HMG CoA Reductase, the control point for cholesterol synthesis. The second image shows the initial results page, which includes the EC number.

In this notebook, we will focus on trypsin, an enzyme produced by your pancreas that helps to digest proteins in your small intestine. It is a member of family called serine hydrolases. Members of this family use water to break bonds in proteins, lipids and carbohydrates. They are involved in digestion, cell signalling and blood clotting. The EC# for trypsin is 3.4.21.4. Here is the meaning of each level of the hierarchy:

3 - Hydrolase
3.4 - acting on peptide bonds
3.4.21 - serine endopeptidase
3.4.21.4 - trypsin

This notebook is intended to help users find ligands for use with docking studies. We are looking for ligands that will bind to trypsin, with the intention of seeing if they will bind to other serine hydrolases that might be interesting. Here are the steps we will follow in the process, all of which will employ the rcsbsearchapi package.

1. Find PDB structures of a given Enzyme Commission class.
2. Select those structures that contain bound small molecules with molecular weights between 300 and 600.
3. Output a list of those ligands
4. Save the ligand structures to the "ligands_for_EC_class_#" folder.

In [2]:
# Import the components of rcsbsearchapi needed for this search
from rcsbapi.search import AttributeQuery as attrs

1. A ligand is any molecule that binds to a protein (including enzymes) to influence its function, while an enzyme is a protein that catalyzes chemical reactions, often by binding to ligands such as substrates.

2. The ligand’s main role is binding, whereas the enzyme’s main role is catalysis.

3. Enzymes are significantly bigger than ligands, both in terms of physical size and molecular weight

4. Ligands are much smaller molecules that bind to enzymes (or other proteins). Ligands can be small organic molecules, ions, or even small peptides, but they are almost always much smaller than the enzyme itself. For example, glucose (a common ligand for enzymes) has a molecular weight of about 180 Da, which is orders of magnitude smaller than a typical enzyme.

# Making queries

To make a query with rcsbsearchapi you first must know what you are looking for. I find it helpful to actually write this out by hand sometimes. Here are the characteristics I am looking for in ligands that bind to a specific Enzyme Commission Class of a protein.

1. EC Class. I will focus on the EC class for trypsin, 3.4.21.4, but any class should work.
2. Ligands. I am looking for ligands that are larger than a single atom (e.g., potassium ion) or a buffer molecule (phosphate), but of a size that consists of 10-30 heavy atoms, so I will aim for a molecular weight between 300 and 800.

Please note that you can use this interface to search for dozens of attributes associated with a PDB entry. The attribute that we will use to look for proteins that have the EC# = 3.4.21.4 is rcsb_polymer_entity.rcsb_ec_lineage.id. Other searchable attributes include the abbreviated journal title for the primary citation, rcsb_primary_citation.rcsb_journal_abbrev, the method used to determine the structure exptl.method, or specific molecules that are part of PDB entries pdbx_reference_molecule.class.

A chemical component in a PDB (Protein Data Bank) structure refers to any distinct chemical entity present in the structure, including:

- Standard and modified amino acids (building blocks of proteins)
- Standard and modified nucleotides (building blocks of DNA/RNA)
- Small molecule ligands (such as drugs, cofactors, or inhibitors)
- Metal ions and solvent molecules (like water)

Each chemical component is assigned a unique three-character alphanumeric code (e.g., “GLC” for glucose, “HEM” for heme, “TYL” for Tylenol)

In [5]:
# There will be three components to the query, which will be labeled q1, q2 and q3.

ECnumber = "3.4.21.4"

q1 = attrs("rcsb_polymer_entity.rcsb_ec_lineage.id", "exact_match", ECnumber)       # looking for trypsin structures with EC = 3.4.21.4
q2 = attrs("chem_comp.formula_weight", "greater_or_equal", 300)                     # setting the lower limit for molecular weight
q3 = attrs("chem_comp.formula_weight", "less_or_equal", 800)                        # setting the lower limit for molecular weight

query = q1 & q2 & q3                     # query (after combining with &) is a query object-it represents the search but does not execute it.

resultL = list(query())                  # Executes the combined query and returns an iterator of results

print(resultL[0:10])
print(f"There are {len(resultL)} trypsin structures that contain ligands in the RCSB PDB.")

['1AQ7', '1AUJ', '1AZ8', '1BJV', '1BTW', '1BTX', '1BTZ', '1C1S', '1C1T', '1C2D']
There are 187 trypsin structures that contain ligands in the RCSB PDB.


# Finding the ligands

This query provided the list of the PDB entries for trypsins (EC # 3.4.21.4) that contain ligands between 300 and 800 molecular weight. We printed the first 10 of these results using print(resultL[0:10]).

Why would we choose to have ligands with molecular weights between 300 and 800? We are interested in molecules that are large enough to bind to and fill up the active site of trypsin. Small molecules with molecular weights less than 300 are likely to be individual ions (K+ or Na+). Molecules with molecular weights greater than 800 will fill more than the active site of an enzyme.

Here is an image of one of the ligands from the search that we're going to learn to download. It is identified in the Protein Data Bank as 13U. As a group we will look at some of the features that make this ligand interesting.

The last statement in the previous cell

print("There are", len(resultL), "trypsin structures that contain ligands in the RCSB PDB.")

tells us how many PDB entries have ligands of that size. The default return item for the query is structure, which provides the four character alphanumeric entry for the full structure in the PDB. We want to identify and download the ligands that are bound to these PDB structures, so we need to switch return types.

To get the ligand, instead of returning the structure (Enzyme + ligand), we will request a return type of mol_definition which will then return the three character alphanumeric entry for the ligand. Other possible return types are polymer entities, assemblies, and non-polymer entities.

In [6]:
molResultL = list(query("mol_definition"))
print(f"There are {len(molResultL)} ligands for EC number {ECnumber} in the list.")
print(molResultL[0:10])

There are 119 ligands for EC number 3.4.21.4 in the list.
['0CA', '0CB', '0G6', '0IV', '0KV', '0ZG', '0ZW', '0ZX', '0ZY', '10U']


**Where can we go to download the ligand files?**

To download the files for ligands bound to trypsin in the RCSB PDB, execute the two cells above for finding the trypsin ligands. This will reset the results to the ones we want.

Once this is done, the next step is to determine exactly what we want to download. These ligand files in the PDB are avaiable for download in several formats. A full list and description can be found in the Small Molecule File table on the RCSB PDB File Download Services page, which is pasted in here.

From this table, we want the ideal coordinate ligand files in mol2 format, which we will later convert to another format called pdbqt for docking.

# How do we download the ligand files?

There are several options for downloading files - we will use a Python libary called requests. In the following cells, we will import the library, requests, download a single file from the RCSB PDB using the requests.get function, and check to make sure the file downloaded properly to the ligands folder. If that is successful, we'll use a for loop to download all of the files from our molResultL list to the ligands folder.

In [7]:
import requests
import os

In [8]:
# Download one of the files from our list: 11U.mol2

res11U_mol2 = requests.get('https://files.rcsb.org/ligands/download/11U_ideal.sdf')

In [10]:
# check to see that the file downloaded properly. A status code of 200 means everything is okay.

res11U_mol2.status_code

200

In [11]:
# Make a ligands folder for our clients. 
os.makedirs("ligands", exist_ok=True)

with open("ligands/res11U_mol2", 'w+') as file:
    file.write(res11U_mol2.text)

In [12]:
file1 = open("ligands/res11U_mol2", 'r')
file_text = file1.read()
print(file_text)

11U
  CCTOOLS-0919241032

 59 61  0  0  1  0  0  0  0  0999 V2000
    2.4220    0.4070    0.3360 C   0  0  0  0  0
    2.0060   -0.6420    0.7800 O   0  0  0  0  0
    3.8690    0.5350   -0.0630 C   0  0  0  0  0
    4.5590   -0.7380    0.1810 N   0  0  0  0  0
    5.9760   -0.6510   -0.1970 C   0  0  0  0  0
    6.7790   -0.0680    0.9670 C   0  0  0  0  0
    8.2550    0.0240    0.5730 C   0  0  0  0  0
    8.7810   -1.3740    0.2400 C   0  0  0  0  0
    7.9780   -1.9570   -0.9250 C   0  0  0  0  0
    6.5020   -2.0480   -0.5310 C   0  0  0  0  0
    1.5890    1.4580    0.1990 N   0  0  0  0  0
    0.1600    1.4690    0.5420 C   0  0  0  0  0
   -0.5770    0.4580   -0.2980 C   0  0  0  0  0
    0.0270   -0.2120   -1.1080 O   0  0  0  0  0
   -0.3710    2.8880    0.2460 C   0  0  0  0  0
    0.9140    3.7560    0.2780 C   0  0  0  0  0
    1.9600    2.7850   -0.3250 C   0  0  0  0  0
   -1.9070    0.2990   -0.1490 N   0  0  0  0  0
   -2.6230   -0.6850   -0.9660 C   0  0  0  0  0
   

# Downloading all of the ligands using a for loop

Now that we know that our process functions, we will use a for loop to download the entire list of ligands (all 112) in a single cell. Here are the steps we will take:

1. Define a variable, baseUrl, for the URL where the ligand files are located. The URL only lacks the specific name of the ligand file.
2. Set up a for loop to go through each of the items (as ChemID) in the molResultL list that was generated above.
3. Assign the filename based on a variable (the 3-letter name of the ligand as ChemID followed by _ideal.mol2) to the variable cFile.
4. Assign the full URL (as cFileUrl) that we want to use to download the data from the RCSB PDB API. Notice that the URL will consist of the baseUrl (defined in the first line of the cell) followed by the name of the file we just defined, which is now assigned to the variable, cFile.
5. Tell the notebook that we want the file (CFileLocal) to be written to the ligands folder, using the os.path command.
6. Use the API call via requests.get to download the data from the RCSB PDB.
7. Write the file using the with open function.

If all goes according to plan, this should download all of the ligands on our list to the ligands folder.

In [13]:
baseUrl = "https://files.rcsb.org/ligands/download/"

for ChemID in molResultL:
    cFile = f"{ChemID}_ideal.sdf"
    cFileUrl = baseUrl + cFile
    cFileLocal = "ligands/" + cFile
    response = requests.get(cFileUrl)
    with open(cFileLocal, "w+") as file:
        file.write(response.text)